## cifar10-cnn example
### keras example: cifar10_cnn.py

import

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


hyperparameters

In [10]:
batch_size = 32
num_classes = 10
epochs = 200
data_augmentation = False

load dataset

In [20]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:100]
y_test = y_test[:100]


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train:', y_train.shape)

x_train shape: (1000, 32, 32, 3)
1000 train samples
100 test samples
y_train: (1000, 10)


model

In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

prepare for trining

In [21]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

training

In [22]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Not using data augmentation.
Train on 1000 samples, validate on 100 samples
Epoch 1/200
1000/1000 [==============================] - 9s - loss: 2.3889 - acc: 0.1320 - val_loss: 2.2452 - val_acc: 0.1600
Epoch 2/200
1000/1000 [==============================] - 9s - loss: 2.2042 - acc: 0.1820 - val_loss: 2.0966 - val_acc: 0.2900
Epoch 3/200
1000/1000 [==============================] - 9s - loss: 2.0560 - acc: 0.2450 - val_loss: 2.0543 - val_acc: 0.2800
Epoch 4/200
1000/1000 [==============================] - 9s - loss: 1.9785 - acc: 0.2650 - val_loss: 1.9938 - val_acc: 0.2400
Epoch 5/200
1000/1000 [==============================] - 8s - loss: 1.8894 - acc: 0.3330 - val_loss: 1.8279 - val_acc: 0.3800
Epoch 6/200
1000/1000 [==============================] - 9s - loss: 1.7842 - acc: 0.3650 - val_loss: 1.8303 - val_acc: 0.3600
Epoch 7/200
1000/1000 [==============================] - 9s - loss: 1.7096 - acc: 0.4050 - val_loss: 1.9963 - val_acc: 0.2800
Epoch 8/200
1000/1000 [===================

KeyboardInterrupt: 